In [11]:
import numpy as np
import torch
from torch.distributions.dirichlet import Dirichlet
from torch.distributions import Independent

In [24]:
random_x = torch.from_numpy(np.random.randn(30,10))

In [25]:
head = Dirichlet(random_x)

In [26]:
out = head.sample()

In [27]:
out.shape

torch.Size([30, 10])

In [28]:
out.sum(dim=-1)

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000], dtype=torch.float64)